In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import mne
import matplotlib.pyplot as plt
import pyvista
import ipywidgets
import ipyevents
import pyvistaqt
import yasa

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_curve, auc
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

import scipy.signal as signal
from scipy.signal import hilbert
from scipy.signal import stft

In [2]:
%matplotlib qt

### One-input CNN model

In [3]:
def build_cnn_model_downsampled(input_shape=(300,1)):

    # linear embedding layer
    input_layer = tf.keras.layers.Input(shape=input_shape)

    # Three convolutional blocks (like having three pattern detectors)

    # First convolution block, kernel size of 5
    padded1 = tf.keras.layers.ZeroPadding1D(padding=2)(input_layer)
    conv1 = tf.keras.layers.Conv1D(filters=10, kernel_size=5, strides=1, padding='valid')(padded1)
    # each filter learns a different type of short-time feature
    # stride of 1, moves one step at a time
    conv1 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv1)
    conv1 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1)
    # K = 2
    conv1 = tf.keras.layers.BatchNormalization()(conv1)

    # Second convolution block, kernel size of 11
    padded2 = tf.keras.layers.ZeroPadding1D(padding=5)(input_layer)
    conv2 = tf.keras.layers.Conv1D(filters=10, kernel_size=11, strides=1, padding='valid')(padded2)
    conv2 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv2)
    conv2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv2)
    conv2 = tf.keras.layers.BatchNormalization()(conv2)

    # Third convolution block, kernel size of 21
    padded3 = tf.keras.layers.ZeroPadding1D(padding=10)(input_layer)
    conv3 = tf.keras.layers.Conv1D(filters=10, kernel_size=21, strides=1, padding='valid')(padded3)
    conv3 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv3)
    conv3 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv3)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)

    # Concatenate the outputs of all blocks
    concatenated = tf.keras.layers.Concatenate()([conv1, conv2, conv3])

    # GRU Layer
    gru = tf.keras.layers.GRU(64)(concatenated)

    # Fully connected (dense) layer
    dense = tf.keras.layers.Dense(64, activation='relu')(gru)
    # add a Dropout layer to prevent overfitting
    #dense = tf.keras.layers.Dropout(0.5)(dense)

    # Two softmax outputs for dual-task classification
    #output_task1 = tf.keras.layers.Dense(2, activation='softmax', name='task1')(dense)
    #output_task2 = tf.keras.layers.Dense(2, activation='softmax', name='task2')(dense)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

    # Create the model
    #model = tf.keras.models.Model(inputs=input_layer, outputs=[output_task1, output_task2])
    model = tf.keras.models.Model(inputs=input_layer, outputs=output)

    # Compile the model
    #model.compile(optimizer='adam', loss={'task1': 'categorical_crossentropy', 'task2': 'categorical_crossentropy'}, metrics={'task1': 'accuracy', 'task2': 'accuracy'})
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Return the compiled model
    return model

In [4]:
def build_2d_cnn_model_downsampled(input_shape=(26, 13, 1)):
    # Input: STFT spectrogram (freq_bins x time_bins x 1)
    input_layer = tf.keras.layers.Input(shape=input_shape)

    # First convolution block, kernel size (3x3)
    padded1 = tf.keras.layers.ZeroPadding2D(padding=(1, 1))(input_layer)
    conv1 = tf.keras.layers.Conv2D(filters=10, kernel_size=(3, 3), strides=(1, 1), padding='valid')(padded1)
    conv1 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv1)
    conv1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv1 = tf.keras.layers.BatchNormalization()(conv1)

    # Second convolution block, kernel size (5x5)
    padded2 = tf.keras.layers.ZeroPadding2D(padding=(2, 2))(input_layer)
    conv2 = tf.keras.layers.Conv2D(filters=10, kernel_size=(5, 5), strides=(1, 1), padding='valid')(padded2)
    conv2 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv2)
    conv2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv2 = tf.keras.layers.BatchNormalization()(conv2)

    # Third convolution block, kernel size (7x7)
    padded3 = tf.keras.layers.ZeroPadding2D(padding=(3, 3))(input_layer)
    conv3 = tf.keras.layers.Conv2D(filters=10, kernel_size=(7, 7), strides=(1, 1), padding='valid')(padded3)
    conv3 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv3)
    conv3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)

    # Concatenate all three conv outputs
    concatenated = tf.keras.layers.Concatenate(axis=-1)([conv1, conv2, conv3])

    # Flatten before passing to RNN
    reshaped = tf.keras.layers.Reshape((-1, concatenated.shape[-1]))(concatenated)

    # GRU layer for temporal modeling over time slices
    gru = tf.keras.layers.GRU(64)(reshaped)

    # Fully connected dense layer
    dense = tf.keras.layers.Dense(64, activation='relu')(gru)

    # Output layer for binary classification
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

    # Define and compile model
    model = tf.keras.models.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model




### Three-input CNN model

In [5]:
def build_multi_input_cnn_model_time():
    
    input_raw = tf.keras.Input(shape=(300, 1), name='raw_input')
    input_filtered = tf.keras.Input(shape=(300, 1), name='filtered_input')
    input_stft = tf.keras.Input(shape=(26, 1), name='stft_input')  
    # 300 time points of raw and filtered EEG
    # 26 time points from the STFT transform

    def conv_branch(input_layer, kernel_sizes=[5, 11, 21]):
        outputs = []
        for k in kernel_sizes:
            pad = k // 2
            x = tf.keras.layers.ZeroPadding1D(padding=pad)(input_layer)
            # applies zero-padding so that the length after convolution stays the same
            x = tf.keras.layers.Conv1D(filters=10, kernel_size=k, strides=1, padding='valid')(x)
            # performs the convolution
            x = tf.keras.layers.LeakyReLU(negative_slope=0.01)(x)
            # applies LeakyReLU 
            x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
            # reduce temporal dimension by 2
            x = tf.keras.layers.BatchNormalization()(x)
            # normalize the output
            outputs.append(x)
        return tf.keras.layers.Concatenate()(outputs)

    # Convolutional branches: each input has its own CNN branch
    branch_raw = conv_branch(input_raw)
    branch_filtered = conv_branch(input_filtered)
    branch_stft = conv_branch(input_stft)

    # Each branch through its own GRU
    # this captures temporal dependencies
    gru_raw = tf.keras.layers.GRU(64)(branch_raw)
    gru_filtered = tf.keras.layers.GRU(64)(branch_filtered)
    gru_stft = tf.keras.layers.GRU(64)(branch_stft)

    # Concatenate GRU outputs (fixed-length vectors)
    # this leads to one 192-dimensional vector
    merged = tf.keras.layers.Concatenate()([gru_raw, gru_filtered, gru_stft])

    # Dense layers
    x = tf.keras.layers.Dense(64, activation='relu')(merged)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Build model
    model = tf.keras.Model(inputs=[input_raw, input_filtered, input_stft], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
def build_multi_input_cnn_model_freq():
    # Inputs
    input_raw = tf.keras.Input(shape=(300, 1), name='raw_input')
    input_filtered = tf.keras.Input(shape=(300, 1), name='filtered_input')
    input_stft = tf.keras.Input(shape=(13, 1), name='stft_input')  

    def conv_branch(input_layer, kernel_sizes=[5, 11, 21]):
        outputs = []
        for k in kernel_sizes:
            pad = k // 2
            x = tf.keras.layers.ZeroPadding1D(padding=pad)(input_layer)
            x = tf.keras.layers.Conv1D(filters=10, kernel_size=k, strides=1, padding='valid')(x)
            x = tf.keras.layers.LeakyReLU(negative_slope=0.01)(x)
            x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            outputs.append(x)
        return tf.keras.layers.Concatenate()(outputs)

    # Convolutional branches
    branch_raw = conv_branch(input_raw)
    branch_filtered = conv_branch(input_filtered)
    branch_stft = conv_branch(input_stft)

    # Each branch through its own GRU
    gru_raw = tf.keras.layers.GRU(64)(branch_raw)
    gru_filtered = tf.keras.layers.GRU(64)(branch_filtered)
    gru_stft = tf.keras.layers.GRU(64)(branch_stft)

    # Concatenate GRU outputs (fixed-length vectors)
    merged = tf.keras.layers.Concatenate()([gru_raw, gru_filtered, gru_stft])

    # Dense layers
    x = tf.keras.layers.Dense(64, activation='relu')(merged)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Build model
    model = tf.keras.Model(inputs=[input_raw, input_filtered, input_stft], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [7]:
def build_multi_input_cnn_model_freq_augmented():
    # Inputs
    input_raw = tf.keras.Input(shape=(360, 1), name='raw_input')
    input_filtered = tf.keras.Input(shape=(360, 1), name='filtered_input')
    input_stft = tf.keras.Input(shape=(16, 1), name='stft_input')  

    def conv_branch(input_layer, kernel_sizes=[5, 11, 21]):
        outputs = []
        for k in kernel_sizes:
            pad = k // 2
            x = tf.keras.layers.ZeroPadding1D(padding=pad)(input_layer)
            x = tf.keras.layers.Conv1D(filters=10, kernel_size=k, strides=1, padding='valid')(x)
            x = tf.keras.layers.LeakyReLU(negative_slope=0.01)(x)
            x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            outputs.append(x)
        return tf.keras.layers.Concatenate()(outputs)

    # Convolutional branches
    branch_raw = conv_branch(input_raw)
    branch_filtered = conv_branch(input_filtered)
    branch_stft = conv_branch(input_stft)

    # Each branch through its own GRU
    gru_raw = tf.keras.layers.GRU(64)(branch_raw)
    gru_filtered = tf.keras.layers.GRU(64)(branch_filtered)
    gru_stft = tf.keras.layers.GRU(64)(branch_stft)

    # Concatenate GRU outputs (fixed-length vectors)
    merged = tf.keras.layers.Concatenate()([gru_raw, gru_filtered, gru_stft])

    # Dense layers
    x = tf.keras.layers.Dense(64, activation='relu')(merged)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Build model
    model = tf.keras.Model(inputs=[input_raw, input_filtered, input_stft], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

### Slow oscillation detection function

In [8]:
def detect_slow_oscillations_times(combined_raw, do_filter=True, do_downsample=False, downsample_rate=100):

    # according to methods from Klinzing et al.(2016)

    data = combined_raw.copy().pick_channels(['Fz'])

    if do_filter:
        data.filter(l_freq=0.16, h_freq=1.25)

    if do_downsample:
        data.resample(downsample_rate)
        
    sfreq = data.info['sfreq']
    channel_data = data.get_data()[0]
    
    # 3. find all positive-to-negative zero-crossings
    
    # zero_crossings = np.where( S!= 0)[0]
    # can also save this somewhere for further detection of spindles
    
    S = np.diff(np.sign(channel_data))
    # np.sign returns an array with 1 (positive), 0 (zero), -1 (negative)
    # np.diff calculates the difference between consecutive elements in an array
    # positive value: transition from negative to positive
    # negative value: transition from positive to negative
    # when it's a zero, means that value stayed the same
    zero_crossings = np.where(S < 0)[0]
    # -2 is when a positive-to-negative zero-crossing occurs
    # goes from 1 to -1 
    # -1 - 1 = -2
    # [0] extracts the actual array
    # extracts the indices of interest from current_data (not S)
    #signs = np.sign(current_data)
    #pos_to_neg = np.where((signs[:-1] > 0) & (signs[1:] < 0))[0]
    # detect +1 to -1
    #neg_to_pos = np.where((signs[:-1] <  0) & (signs[1:] > 0))[0]
    # detect -1 to +1

    # 4. Detect peak potentials in each pair
    slow_oscillations = []
    negative_peaks = []
    positive_peaks = []
    peak_to_peak_amplitudes = []
    candidate_indices = []

    # for loop for each pair
    # to collect all the negative and positive peaks
    # to further apply criteria
    count = 0
    for i in range(0, len(zero_crossings)-1, 1):
        # loop through all the zero_crossings
        # step of 1 (with step of 2, miss some zero_crossings)
        start_idx = zero_crossings[i] + 1
        # assigns index of zero-crossing (representing start of potential SO)
        # to start_idx
        end_idx = zero_crossings[i + 1] + 1
        # assigns index of next zero-crossing (representing end of potential SO)
        # to end_idx

        # find the negative to positive crossing in between
        #mid_crossings = neg_to_pos[(neg_to_pos > start_idx) & (neg_to_pos < end_idx)]

        #if len(mid_crossings) != 1:
            #continue

        #mid_idx = mid_crossings [0]

        #duration = (end_idx - start_idx) / sfreq
        #if not (0.8 <= duration <= 2.0):
  
        
        segment_length = (end_idx - start_idx) / sfreq

        # need to add +1 because of way extract segment later

        # have identified index for the pair
        
        # extract data segment between crossings
        
        # find peaks
        if 0.8 <= segment_length <= 2.0:
            count += 1
            segment = channel_data[start_idx:end_idx]
            positive_peak = np.max(segment)
            negative_peak = np.min(segment)
            peak_to_peak_amplitude = positive_peak - negative_peak

        # store values
            candidate_indices.append((start_idx, end_idx))
            positive_peaks.append(positive_peak)
            negative_peaks.append(negative_peak)
            peak_to_peak_amplitudes.append(peak_to_peak_amplitude)

    # calculate mean values for comparison
    #mean_negative_peak = np.mean(negative_peaks)
    # mean_negative_peak = np.mean(negative_peaks) if negative_peaks else 0
    #mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes)
    # mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes) if peak_to_peak_amplitudes else 0

    negative_peak_threshold = np.percentile(negative_peaks, 25)
    # keep lowest negative peaks (under the 25th percentile)
    peak_to_peak_amplitude_threshold = np.percentile(peak_to_peak_amplitudes, 75)
    # keep largest peak-to-peak amplitude (over 75th percentile)

    for (start_idx, end_idx), negative_peak, peak_to_peak_amplitude in zip(candidate_indices, negative_peaks, peak_to_peak_amplitudes):
        if peak_to_peak_amplitude >= peak_to_peak_amplitude_threshold and negative_peak <= negative_peak_threshold:
            slow_oscillations.append((start_idx / sfreq, end_idx / sfreq))
            
    return slow_oscillations
    # returns a list of tuples, in which each tuple represents the start and end times of
    # a detected slow oscillation

def detect_slow_oscillations_peaks(combined_raw, do_filter=True, do_downsample=True, downsample_rate=100):

    # according to methods from Klinzing et al.(2016)

    data = combined_raw.copy().pick_channels(['Fz'])

    if do_filter:
        data.filter(l_freq=0.16, h_freq=1.25)

    if do_downsample:
        data.resample(downsample_rate)
        
    sfreq = data.info['sfreq']
    channel_data = data.get_data()[0]
    
    # 3. find all positive-to-negative zero-crossings
    
    # zero_crossings = np.where( S!= 0)[0]
    # can also save this somewhere for further detection of spindles
    
    S = np.diff(np.sign(channel_data))
    # np.sign returns an array with 1 (positive), 0 (zero), -1 (negative)
    # np.diff calculates the difference between consecutive elements in an array
    # positive value: transition from negative to positive
    # negative value: transition from positive to negative
    # when it's a zero, means that value stayed the same
    zero_crossings = np.where(S < 0)[0]
    # -2 is when a positive-to-negative zero-crossing occurs
    # goes from 1 to -1 
    # -1 - 1 = -2
    # [0] extracts the actual array
    # extracts the indices of interest from current_data (not S)


    # 4. Detect peak potentials in each pair
    slow_oscillations = []
    slow_oscillations_peaks = []
    negative_peaks = []
    positive_peaks = []
    peak_to_peak_amplitudes = []
    candidate_indices =  []

    # for loop for each pair
    # to collect all the negative and positive peaks
    # to further apply criteria
    count = 0
    for i in range(0, len(zero_crossings) - 1, 1):
        # loop through all the zero_crossings
        # step of 1 (with step of 2, miss some zero_crossings)
        start_idx = zero_crossings[i] + 1
        # assigns index of zero-crossing (representing start of potential SO)
        # to start_idx
        end_idx = zero_crossings[i + 1] + 1
        # assigns index of next zero-crossing (representing end of potential SO)
        # to end_idx
        segment_length = (end_idx - start_idx) / sfreq

        # need to add +1 because of way extract segment later

        # have identified index for the pair
        
        # extract data segment between crossings
        
        # find peaks
        if 0.8 <= segment_length <= 2.0:
            count += 1
            segment = channel_data[start_idx:end_idx]
            positive_peak = np.max(segment)
            negative_peak = np.min(segment)
            peak_to_peak_amplitude = positive_peak - negative_peak

        # store values
            candidate_indices.append((start_idx, end_idx))
            positive_peaks.append(positive_peak)
            negative_peaks.append(negative_peak)
            peak_to_peak_amplitudes.append(peak_to_peak_amplitude)

    # calculate mean values for comparison
    #mean_negative_peak = np.mean(negative_peaks)
    # mean_negative_peak = np.mean(negative_peaks) if negative_peaks else 0
    #mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes)
    # mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes) if peak_to_peak_amplitudes else 0

    negative_peak_threshold = np.percentile(negative_peaks, 25)
    peak_to_peak_amplitude_threshold = np.percentile(peak_to_peak_amplitudes, 75)

    for (start_idx, end_idx), negative_peak, peak_to_peak_amplitude in zip(candidate_indices, negative_peaks, peak_to_peak_amplitudes):
        if peak_to_peak_amplitude >= peak_to_peak_amplitude_threshold and negative_peak <= negative_peak_threshold:
            slow_oscillations.append((start_idx / sfreq, end_idx / sfreq))
            slow_oscillations_peaks.append((negative_peak, positive_peak))

            
    return slow_oscillations
    # returns a list of tuples, in which each tuple represents the start and end times of
    # a detected slow oscillation

### Epochs function

In [9]:
def create_fixed_length_epochs(raw, duration=3.0, overlap=0.0, preload=True, reject_by_annotation=False):
    #raw_zero = raw.copy().crop(tmin=raw.times[0], tmax=raw.times[-1], include_tmax=True)
    # because otherwise does not start at 0
    return mne.make_fixed_length_epochs(
        #raw_zero,
        raw,
        duration=duration,
        overlap=overlap,
        preload=preload,
        reject_by_annotation=reject_by_annotation
    )
# function mne.make_fixed_length_epochs takes into account the sampling frequency of the data

# because can change the overlap in the previous function
# can also change this function


def label_so_epochs(epochs, so_starts, so_ends, epoch_length_sec=3.0):

    epoch_starts = np.arange(len(epochs)) * epoch_length_sec
    # new np array with the start time of each epoch
    # epoch_starts[i] is the start time of each epoch

    epoch_labels = np.zeros(len(epochs), dtype=int)
    # initialize all the labels as 0 initially

    for start, end in zip(so_starts, so_ends):
        # loop through the start and end times of detected spindles by YASA
        for i, epoch_start in enumerate(epoch_starts):
            # loop through the one-second epochs that are not labelled yet
            epoch_end = epoch_start + epoch_length_sec
            # for each epoch, calculate the epoch end time
            # which is epoch_start + length of epoch
            # so now have the time range of each epoch
            if (start < epoch_end) and (end > epoch_start):
                # if the spindle started before the epoch ends
                # and the spindle ended after the epoch started
                epoch_labels[i] = 1
                
    return epoch_labels

def label_so_epochs_strict(epochs, so_starts, so_ends, epoch_length_sec=3.0):
    epoch_starts = np.arange(len(epochs)) * epoch_length_sec
    epoch_labels = np.zeros(len(epochs), dtype=int)

    for so_start, so_end in zip(so_starts, so_ends):
        so_duration = so_end - so_start
        required_overlap = 0.8 * so_duration  
        # only label 1 if epoch contains 80% of the SO duration

        for i, epoch_start in enumerate(epoch_starts):
            epoch_end = epoch_start + epoch_length_sec

            # Calculate overlap between SO and epoch
            overlap_start = max(so_start, epoch_start)
            overlap_end = min(so_end, epoch_end)
            overlap_duration = overlap_end - overlap_start

            if overlap_duration >= required_overlap:
                epoch_labels[i] = 1

    return epoch_labels

def label_so_epochs_moderate(epochs, so_starts, so_ends, epoch_length_sec=3.0):
    epoch_starts = np.arange(len(epochs)) * epoch_length_sec
    epoch_labels = np.zeros(len(epochs), dtype=int)

    for so_start, so_end in zip(so_starts, so_ends):
        so_duration = so_end - so_start
        required_overlap = 0.5 * so_duration  
        # only label 1 if epoch contains 80% of the SO duration

        for i, epoch_start in enumerate(epoch_starts):
            epoch_end = epoch_start + epoch_length_sec

            # Calculate overlap between SO and epoch
            overlap_start = max(so_start, epoch_start)
            overlap_end = min(so_end, epoch_end)
            overlap_duration = overlap_end - overlap_start

            if overlap_duration >= required_overlap:
                epoch_labels[i] = 1

    return epoch_labels



In [ ]:
# def label_so_epochs(epochs, so_starts, so_ends):
#     sfreq = epochs.info['sfreq']
#     epoch_starts = epochs.events[:, 0] / sfreq
#     # gives the sample index at which each epoch starts
#     # divides by sfreq converts these to start times in seconds
#     epoch_length_sec = epochs.get_data().shape[2] / sfreq
#     # gives the duration of each epoch

#     epoch_labels = np.zeros(len(epochs), dtype=int)
#     # initialize all the labels as 0 initially

#     for start, end in zip(so_starts, so_ends):
#         # loop through the start and end times of detected spindles 
#         for i, epoch_start in enumerate(epoch_starts):
#             # loop through the one-second epochs that are not labelled yet
#             epoch_end = epoch_start + epoch_length_sec
#             # for each epoch, calculate the epoch end time
#             # which is epoch_start + length of epoch
#             # so now have the time range of each epoch
#             if (start < epoch_end) and (end > epoch_start):
#                 # if the spindle started before the epoch ends
#                 # and the spindle ended after the epoch started
#                 epoch_labels[i] = 1
                
#     return epoch_labels

### Data augmentation

In [10]:
def augment_so_epochs_with_jitter_and_noise(epochs, labels, sfreq, 
                                               blank_duration=0.3,
                                               jitter_max_shift_sec=0.2, 
                                               noise_std=0.01,
                                               n_augments_per_so=2):
    
    blank_samples = int(blank_duration * sfreq)
    jitter_max_shift = int(jitter_max_shift_sec * sfreq)
    n_samples, n_channels = epochs.shape[1], epochs.shape[2]

    # Create blank padding of zeros with shape (blank_samples, n_channels)
    blank = np.zeros((blank_samples, n_channels))

    padded_epochs = []
    padded_labels = []

    for epoch, label in zip(epochs, labels):
        # Concatenate blank before and after along time axis (axis=0 in this case of blank)
        padded = np.concatenate([blank, epoch, blank], axis=0)  # shape: (n_samples + 2*blank_samples, n_channels)
        padded_epochs.append(padded)
        padded_labels.append(label)

        if label == 1:
            for _ in range(n_augments_per_so):
                # Random jitter shift in samples
                shift = np.random.randint(-jitter_max_shift, jitter_max_shift + 1)
                jittered = np.roll(padded, shift=shift, axis=0)  # roll along time axis
                
                # Add Gaussian noise
                noise = np.random.normal(0, noise_std, size=jittered.shape)
                augmented = jittered + noise

                padded_epochs.append(augmented)
                padded_labels.append(1)  # same label for augmented SO epoch

    return np.array(padded_epochs), np.array(padded_labels)


### Pad test data

In [24]:
def pad_test_data(X_test, pad_before=30, pad_after=30):
    n_epochs, n_samples, n_channels = X_test.shape

    # Create zero padding arrays
    pad_start = np.zeros((n_epochs, pad_before, n_channels))
    pad_end = np.zeros((n_epochs, pad_after, n_channels))

    # Concatenate along time axis (axis=1)
    X_test_padded = np.concatenate([pad_start, X_test, pad_end], axis=1)

    return X_test_padded

### Importing data

In [11]:
# file paths
train_file = r"C:\EEG DATA\combined_sets\train_raw.fif"
test_file = r"C:\EEG DATA\combined_sets\test_raw.fif"

# load raw files
train_raw = mne.io.read_raw_fif(train_file, preload=True)
test_raw = mne.io.read_raw_fif(test_file, preload=True)

#train_raw.crop(tmin=train_raw.times[0], include_tmax=True)
#test_raw.crop(tmin=test_raw.times[0], include_tmax=True)

Opening raw data file C:\EEG DATA\combined_sets\train_raw.fif...
Isotrak not found
    Range : 1470000 ... 23295072 =   2940.000 ... 46590.144 secs
Ready.
Reading 0 ... 21825072  =      0.000 ... 43650.144 secs...
Opening raw data file C:\EEG DATA\combined_sets\test_raw.fif...
Isotrak not found
    Range : 825000 ... 17985049 =   1650.000 ... 35970.098 secs
Ready.
Reading 0 ... 17160049  =      0.000 ... 34320.098 secs...


## With raw data

### Slow oscillation detection

In [12]:
so_train_times_raw_downsampled = detect_slow_oscillations_times(train_raw, do_filter=True, do_downsample=True)
so_test_times_raw_downsampled = detect_slow_oscillations_times(test_raw, do_filter=True, do_downsample=True)

so_starts_train_raw_downsampled, so_ends_train_raw_downsampled = zip(*so_train_times_raw_downsampled) if so_train_times_raw_downsampled else([],[])
so_starts_test_raw_downsampled, so_ends_test_raw_downsampled = zip(*so_test_times_raw_downsampled) if so_test_times_raw_downsampled else([],[])

print(len(so_starts_train_raw_downsampled))
print(len(so_ends_train_raw_downsampled))

print(len(so_starts_test_raw_downsampled))
print(len(so_ends_test_raw_downsampled))

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 50 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194

### Downsample

In [13]:
train_raw_downsampled = train_raw.copy().resample(100)
test_raw_downsampled = test_raw.copy().resample(100)

print(train_raw_downsampled.info['sfreq'])
print(test_raw_downsampled.info['sfreq'])

100.0
100.0


### Epoch the data

In [14]:
epochs_train_raw_downsampled = create_fixed_length_epochs(train_raw_downsampled, duration=3.0, overlap=0.0)
epochs_test_raw_downsampled = create_fixed_length_epochs(test_raw_downsampled, duration=3.0, overlap=0.0)
print(epochs_train_raw_downsampled.get_data().shape)
print(epochs_test_raw_downsampled.get_data().shape)

Not setting metadata
14550 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 14550 events and 300 original time points ...
0 bad epochs dropped
Not setting metadata
11440 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 11440 events and 300 original time points ...
0 bad epochs dropped
(14550, 1, 300)
(11440, 1, 300)


In [42]:
%%time

# Train set

epoch_labels_train_raw_downsampled = label_so_epochs(epochs_train_raw_downsampled, so_starts_train_raw_downsampled, so_ends_train_raw_downsampled)

print(f"Train data first 10 SO: {so_starts_train_raw_downsampled[:10]}")
print(f"Train labels first 10 epochs: {epoch_labels_train_raw_downsampled[:10]}")

# Test set

epoch_labels_test_raw_downsampled = label_so_epochs(epochs_test_raw_downsampled, so_starts_test_raw_downsampled, so_ends_test_raw_downsampled)

print(f"\nTest data first 10 SO: {so_starts_test_raw_downsampled[:10]}")
print(f"Test labels first 10 epochs: {epoch_labels_test_raw_downsampled[:10]}")

Train data first 10 SO: (np.float64(7.82), np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(308.33), np.float64(330.51), np.float64(378.52), np.float64(394.11), np.float64(478.54))
Train labels first 10 epochs: [0 0 1 0 0 0 0 0 0 0]

Test data first 10 SO: (np.float64(65.96), np.float64(120.42), np.float64(161.02), np.float64(224.46), np.float64(257.48), np.float64(353.86), np.float64(404.85), np.float64(406.44), np.float64(440.22), np.float64(459.74))
Test labels first 10 epochs: [0 0 0 0 0 0 0 0 0 0]
CPU times: total: 12.5 s
Wall time: 12.5 s


In [93]:
%%time

# Train set

epoch_labels_train_raw_downsampled_strict = label_so_epochs_strict(epochs_train_raw_downsampled, so_starts_train_raw_downsampled, so_ends_train_raw_downsampled)

print(f"Train data first 10 SO: {so_starts_train_raw_downsampled[:10]}")
print(f"Train labels first 10 epochs: {epoch_labels_train_raw_downsampled[:10]}")

# Test set

epoch_labels_test_raw_downsampled_strict = label_so_epochs_strict(epochs_test_raw_downsampled, so_starts_test_raw_downsampled, so_ends_test_raw_downsampled)

print(f"\nTest data first 10 SO: {so_starts_test_raw_downsampled[:10]}")
print(f"Test labels first 10 epochs: {epoch_labels_test_raw_downsampled[:10]}")

Train data first 10 SO: (np.float64(7.82), np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(308.33), np.float64(330.51), np.float64(378.52), np.float64(394.11), np.float64(478.54))
Train labels first 10 epochs: [0 0 1 0 0 0 0 0 0 0]

Test data first 10 SO: (np.float64(65.96), np.float64(120.42), np.float64(161.02), np.float64(224.46), np.float64(257.48), np.float64(353.86), np.float64(404.85), np.float64(406.44), np.float64(440.22), np.float64(459.74))
Test labels first 10 epochs: [0 0 0 0 0 0 0 0 0 0]
CPU times: total: 33.1 s
Wall time: 33.3 s


In [15]:
%%time

# Train set

epoch_labels_train_raw_downsampled_moderate = label_so_epochs_moderate(epochs_train_raw_downsampled, so_starts_train_raw_downsampled, so_ends_train_raw_downsampled)

print(f"Train data first 10 SO: {so_starts_train_raw_downsampled[:10]}")
print(f"Train labels first 10 epochs: {epoch_labels_train_raw_downsampled_moderate[:10]}")

# Test set

epoch_labels_test_raw_downsampled_moderate = label_so_epochs_moderate(epochs_test_raw_downsampled, so_starts_test_raw_downsampled, so_ends_test_raw_downsampled)

print(f"\nTest data first 10 SO: {so_starts_test_raw_downsampled[:10]}")
print(f"Test labels first 10 epochs: {epoch_labels_test_raw_downsampled_moderate[:10]}")

Train data first 10 SO: (np.float64(7.82), np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(308.33), np.float64(330.51), np.float64(378.52), np.float64(394.11), np.float64(478.54))
Train labels first 10 epochs: [0 0 1 0 0 0 0 0 0 0]

Test data first 10 SO: (np.float64(65.96), np.float64(120.42), np.float64(161.02), np.float64(224.46), np.float64(257.48), np.float64(353.86), np.float64(404.85), np.float64(406.44), np.float64(440.22), np.float64(459.74))
Test labels first 10 epochs: [0 0 0 0 0 0 0 0 0 0]
CPU times: total: 1min 16s
Wall time: 1min 18s


In [16]:
# Train set
train_labels = epoch_labels_train_raw_downsampled
train_counts = np.bincount(train_labels)
train_total = len(train_labels)

print("\nTrain label distribution:")
for label, count in enumerate(train_counts):
    proportion = count / train_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

# Test set
test_labels = epoch_labels_test_raw_downsampled
test_counts = np.bincount(test_labels)
test_total = len(test_labels)

print("\nTest label distribution:")
for label, count in enumerate(test_counts):
    proportion = count / test_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")


NameError: name 'epoch_labels_train_raw_downsampled' is not defined

In [45]:

# Train set
train_labels = epoch_labels_train_raw_downsampled_strict
train_counts = np.bincount(train_labels)
train_total = len(train_labels)

print("\nTrain label distribution:")
for label, count in enumerate(train_counts):
    proportion = count / train_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

# Test set
test_labels = epoch_labels_test_raw_downsampled_strict
test_counts = np.bincount(test_labels)
test_total = len(test_labels)

print("\nTest label distribution:")
for label, count in enumerate(test_counts):
    proportion = count / test_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")



Train label distribution:
Label 0: count = 12185, proportion = 83.75%
Label 1: count = 2365, proportion = 16.25%

Test label distribution:
Label 0: count = 10011, proportion = 87.51%
Label 1: count = 1429, proportion = 12.49%


In [17]:

# Train set
train_labels = epoch_labels_train_raw_downsampled_moderate
train_counts = np.bincount(train_labels)
train_total = len(train_labels)

print("\nTrain label distribution:")
for label, count in enumerate(train_counts):
    proportion = count / train_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")

# Test set
test_labels = epoch_labels_test_raw_downsampled_moderate
test_counts = np.bincount(test_labels)
test_total = len(test_labels)

print("\nTest label distribution:")
for label, count in enumerate(test_counts):
    proportion = count / test_total
    print(f"Label {label}: count = {count}, proportion = {proportion:.2%}")



Train label distribution:
Label 0: count = 11622, proportion = 79.88%
Label 1: count = 2928, proportion = 20.12%

Test label distribution:
Label 0: count = 9641, proportion = 84.27%
Label 1: count = 1799, proportion = 15.73%


### Data augmentation

In [19]:
# Reshape arrays

epochs_train_np_raw_downsampled = np.array(epochs_train_raw_downsampled).reshape(len(epochs_train_raw_downsampled), -1, 1)


In [20]:
augmented_epochs_train_raw_downsampled, augmented_epoch_labels_train_raw_downsampled_moderate = augment_so_epochs_with_jitter_and_noise(epochs_train_np_raw_downsampled, epoch_labels_train_raw_downsampled_moderate, sfreq=100, 
                                             blank_duration=0.3,
                                             jitter_max_shift_sec=0.2, 
                                             noise_std=0.01,
                                             n_augments_per_so=2)



In [131]:
print(f"Train data first 10 SO: {so_starts_train_raw_downsampled[:10]}")
print(f"Train labels first 10 epochs: {epoch_labels_train_raw_downsampled[:10]}")

Train data first 10 SO: (np.float64(7.82), np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(308.33), np.float64(330.51), np.float64(378.52), np.float64(394.11), np.float64(478.54))
Train labels first 10 epochs: [0 0 1 0 0 0 0 0 0 0]


In [21]:
print("Train data:")
print("\nOriginal shape:", epochs_train_np_raw_downsampled.shape)
print("Augmented shape:", augmented_epochs_train_raw_downsampled.shape)
print("Original class balance:", np.bincount(epoch_labels_train_raw_downsampled_moderate))
print("Augmented class balance:", np.bincount(augmented_epoch_labels_train_raw_downsampled_moderate))


Train data:

Original shape: (14550, 300, 1)
Augmented shape: (20406, 360, 1)
Original class balance: [11622  2928]
Augmented class balance: [11622  8784]


### Prepare EEG data for CNN Inputs

#### X and y train and test sets

In [22]:
# Reshape arrays

#epochs_train_np_raw_downsampled = np.array(epochs_train_raw_downsampled).reshape(len(epochs_train_raw_downsampled), -1, 1)
# number of epochs N, sampling frequency (time dimension automatically inferred), channel dimension
epochs_test_np_raw_downsampled = np.array(epochs_test_raw_downsampled).reshape(len(epochs_test_raw_downsampled), -1, 1)

In [25]:
X_test_raw_downsampled_padded = pad_test_data(epochs_test_np_raw_downsampled, pad_before=30, pad_after=30)
print(X_test_raw_downsampled_padded.shape) 

(11440, 360, 1)


In [26]:
# Define X and y sets
                                                                 
X_train_raw_downsampled = augmented_epochs_train_raw_downsampled
y_train_raw_downsampled = augmented_epoch_labels_train_raw_downsampled_moderate

X_test_raw_downsampled = X_test_raw_downsampled_padded
y_test_raw_downsampled = epoch_labels_test_raw_downsampled_moderate

# Print shapes

print(f"X_train shape: {X_train_raw_downsampled.shape}")
print(f"y_train shape: {y_train_raw_downsampled.shape}")

print(f"\nX_test shape: {X_test_raw_downsampled.shape}")
print(f"y_test shape: {y_test_raw_downsampled.shape}")

X_train shape: (20406, 360, 1)
y_train shape: (20406,)

X_test shape: (11440, 360, 1)
y_test shape: (11440,)


#### Train the model

In [27]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,               
    restore_best_weights=True 
)
# stop after 5 epochs with no improvement

In [28]:
# show model architecture
input_shape = (360, 1)
model_raw_downsampled_so = build_cnn_model_downsampled(input_shape)
model_raw_downsampled_so.summary()

C:\Users\roseb\anaconda3\envs\msc_research_project\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 360, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d      │ (None, 364, 1)    │          0 │ input_layer[0][0] │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_1    │ (None, 370, 1)    │          0 │ input_layer[0][0] │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_2    │ (None, 380, 1)    │          0 │ input_layer[0][0] │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 360, 10)   │         60 │ zero_padding1d[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 360, 10)   │        120 │ zero_padding1d_1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 360, 10)   │        220 │ zero_padding1d_2… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 360, 10)   │          0 │ conv1d[0][0]      │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 360, 10)   │          0 │ conv1d_1[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 360, 10)   │          0 │ conv1d_2[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 180, 10)   │          0 │ leaky_re_lu[0][0] │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 180, 10)   │          0 │ leaky_re_lu_1[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 180, 10)   │          0 │ leaky_re_lu_2[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 180, 10)   │         40 │ max_pooling1d[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 180, 10)   │         40 │ max_pooling1d_1[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 180, 10)   │         40 │ max_pooling1d_2[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 180, 30)   │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 23,177 (90.54 KB)

 Trainable params: 23,117 (90.30 KB)

 Non-trainable params: 60 (240.00 B)

In [29]:
%%time

training_info_raw_downsampled_so = model_raw_downsampled_so.fit(X_train_raw_downsampled, y_train_raw_downsampled, validation_split=0.2, epochs=30, batch_size=128, callbacks=[early_stop])

Epoch 1/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 37s 226ms/step - accuracy: 0.6053 - loss: 0.6593 - val_accuracy: 0.7903 - val_loss: 0.4887
Epoch 2/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 29s 226ms/step - accuracy: 0.7682 - loss: 0.4858 - val_accuracy: 0.8413 - val_loss: 0.3976
Epoch 3/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 29s 226ms/step - accuracy: 0.8110 - loss: 0.4111 - val_accuracy: 0.8432 - val_loss: 0.3672
Epoch 4/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 28s 221ms/step - accuracy: 0.8541 - loss: 0.3122 - val_accuracy: 0.8986 - val_loss: 0.2432
Epoch 5/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 29s 224ms/step - accuracy: 0.9027 - loss: 0.2249 - val_accuracy: 0.9096 - val_loss: 0.2097
Epoch 6/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 28s 221ms/step - accuracy: 0.9117 - loss: 0.2081 - val_accuracy: 0.9143 - val_loss: 0.2058
Epoch 7/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 29s 229ms/step - accuracy: 0.9106 - loss: 0.2066 - val_accuracy: 0.9182 - val_loss: 0.2027
Epoch 8/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 29s 226ms/step - accuracy: 0.9103 - loss: 0

#### Plot the training history

In [151]:
def plot_training_history(training_info):
    fig, axs = plt.subplots(1, 2, figsize=(16, 5))
    axs[0].plot(training_info.history['loss'], label="training set")
    axs[0].plot(training_info.history['val_loss'], label="validation set")
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].grid(True)
    axs[0].legend()
    try:
        axs[1].plot(training_info.history['accuracy'], label="training set")
        axs[1].plot(training_info.history['val_accuracy'], label="validation set")
        axs[1].set_xlabel("Epoch")
        axs[1].set_ylabel("Accuracy")
        axs[1].grid(True)
        axs[1].legend()
    except:
        pass
  
    fig.suptitle("Training history for one-input CNN model with raw EEG data for SO detection", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

plot_training_history(training_info_raw_downsampled_so)

In [152]:
model_raw_downsampled_so.evaluate(X_test_raw_downsampled, y_test_raw_downsampled)

470/470 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.9060 - loss: 0.2042


[0.24060454964637756, 0.8816996812820435]

In [30]:
y_pred = model_raw_downsampled_so.predict(X_test_raw_downsampled)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test_raw_downsampled, y_pred_labels))
print(classification_report(y_test_raw_downsampled, y_pred_labels))

358/358 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step
[[8384 1257]
 [ 521 1278]]
              precision    recall  f1-score   support

           0       0.94      0.87      0.90      9641
           1       0.50      0.71      0.59      1799

    accuracy                           0.84     11440
   macro avg       0.72      0.79      0.75     11440
weighted avg       0.87      0.84      0.85     11440



In [154]:
# Get predictions
y_pred = model_raw_downsampled_so.predict(X_test_raw_downsampled)
y_pred_labels = (y_pred > 0.5).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test_raw_downsampled, y_pred_labels)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Classification report as a dataframe
report = classification_report(y_test_raw_downsampled, y_pred_labels, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Confusion matrix plotted as heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix for SO Detection \nusing a one-input CNN with raw EEG data")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()

# Classification report as a table
fig, ax = plt.subplots(figsize=(10, 4))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=report_df.round(2).values,
                 colLabels=report_df.columns,
                 rowLabels=report_df.index,
                 cellLoc='center',
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)
plt.title("Classification Report for SO Detection \nusing a one-input CNN with raw EEG data", fontsize=14)
plt.tight_layout()
plt.show()


470/470 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step


In [155]:
# Flatten in case y_pred has shape (n_samples, 1)
y_pred_proba = y_pred.ravel()

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test_raw_downsampled, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic for SO Detection \nusing a one-input CNN with raw EEG data")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

## With filtered data

In [31]:
# Apply bandpass filter between 0.16 and 1.25 Hz
# to compare performance of filtered dataset to unfiltered one
train_filtered_downsampled = train_raw.copy().filter(l_freq=0.16, h_freq=1.25)
test_filtered_downsampled = test_raw.copy().filter(l_freq=0.16, h_freq=1.25)

# Downsample to 100 Hz
train_filtered_downsampled.resample(100)
test_filtered_downsampled.resample(100)

Filtering raw data in 73 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)

Filtering raw data in 50 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper pass

<Raw | test_raw.fif, 1 x 3432010 (34320.1 s), ~26.2 MiB, data loaded>

### Slow oscillation detection

In [32]:
so_train_times_filtered_downsampled = detect_slow_oscillations_times(train_filtered_downsampled, do_filter=False, do_downsample=False)
so_test_times_filtered_downsampled = detect_slow_oscillations_times(test_filtered_downsampled, do_filter=False, do_downsample=False)
# since filtering and downsampling before, do not filter and downsample again in function

so_starts_train_filtered_downsampled, so_ends_train_filtered_downsampled = zip(*so_train_times_filtered_downsampled) if so_train_times_filtered_downsampled else([],[])
so_starts_test_filtered_downsampled, so_ends_test_filtered_downsampled = zip(*so_test_times_filtered_downsampled) if so_test_times_filtered_downsampled else([],[])

print(len(so_starts_train_filtered_downsampled))
print(len(so_ends_train_filtered_downsampled))

print(len(so_starts_test_filtered_downsampled))
print(len(so_ends_test_filtered_downsampled))

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
3610
3610
2150
2150


### Epoch the data

In [33]:
epochs_train_filtered_downsampled = create_fixed_length_epochs(train_filtered_downsampled, duration=3.0, overlap=0.0)
epochs_test_filtered_downsampled = create_fixed_length_epochs(test_filtered_downsampled, duration=3.0, overlap=0.0)
print(epochs_train_filtered_downsampled.get_data().shape)
print(epochs_test_filtered_downsampled.get_data().shape)

Not setting metadata
14550 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 14550 events and 300 original time points ...
0 bad epochs dropped
Not setting metadata
11440 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 11440 events and 300 original time points ...
0 bad epochs dropped
(14550, 1, 300)
(11440, 1, 300)


### Labels for 3-second epochs

In [34]:
%%time

# Train set

epoch_labels_train_filtered_downsampled_moderate = label_so_epochs_moderate(epochs_train_filtered_downsampled, so_starts_train_filtered_downsampled, so_ends_train_filtered_downsampled)

print(f"Train data first 10 spindles: {so_starts_train_filtered_downsampled[:10]}")
print(f"Train labels first 10 epochs: {epoch_labels_train_filtered_downsampled_moderate[:10]}")

# Test set

epoch_labels_test_filtered_downsampled_moderate = label_so_epochs_moderate(epochs_test_filtered_downsampled, so_starts_test_filtered_downsampled, so_ends_test_filtered_downsampled)

print(f"\nTest data first 10 spindles: {so_starts_test_filtered_downsampled[:10]}")
print(f"Test labels first 10 epochs: {epoch_labels_test_filtered_downsampled_moderate[:10]}")

Train data first 10 spindles: (np.float64(7.82), np.float64(83.41), np.float64(108.08), np.float64(262.01), np.float64(274.78), np.float64(308.33), np.float64(330.51), np.float64(378.52), np.float64(394.11), np.float64(478.54))
Train labels first 10 epochs: [0 0 1 0 0 0 0 0 0 0]

Test data first 10 spindles: (np.float64(65.96), np.float64(120.42), np.float64(161.02), np.float64(224.46), np.float64(257.48), np.float64(353.86), np.float64(404.85), np.float64(406.44), np.float64(440.22), np.float64(459.74))
Test labels first 10 epochs: [0 0 0 0 0 0 0 0 0 0]
CPU times: total: 1min 14s
Wall time: 1min 16s


### Data augmentation

In [35]:
# Reshape arrays

epochs_train_np_filtered_downsampled = np.array(epochs_train_filtered_downsampled).reshape(len(epochs_train_filtered_downsampled), -1, 1)


In [36]:
augmented_epochs_train_filtered_downsampled, augmented_epoch_labels_train_filtered_downsampled_moderate = augment_so_epochs_with_jitter_and_noise(epochs_train_np_filtered_downsampled, epoch_labels_train_filtered_downsampled_moderate, sfreq=100, 
                                             blank_duration=0.3,
                                             jitter_max_shift_sec=0.2, 
                                             noise_std=0.01,
                                             n_augments_per_so=2)

In [37]:
print("Train data:")
print("\nOriginal shape:", epochs_train_np_filtered_downsampled.shape)
print("Augmented shape:", augmented_epochs_train_filtered_downsampled.shape)
print("Original class balance:", np.bincount(epoch_labels_train_filtered_downsampled_moderate))
print("Augmented class balance:", np.bincount(augmented_epoch_labels_train_filtered_downsampled_moderate))


Train data:

Original shape: (14550, 300, 1)
Augmented shape: (20406, 360, 1)
Original class balance: [11622  2928]
Augmented class balance: [11622  8784]


### Prepare EEG data for CNN input

##### X and y train and test sets

In [38]:
epochs_test_np_filtered_downsampled = np.array(epochs_test_filtered_downsampled).reshape(len(epochs_test_filtered_downsampled), -1, 1)

In [39]:
X_test_filtered_downsampled_padded = pad_test_data(epochs_test_np_filtered_downsampled, pad_before=30, pad_after=30)
print(X_test_filtered_downsampled_padded.shape) 

(11440, 360, 1)


In [40]:
# Define X and y sets
                                                                 
X_train_filtered_downsampled = augmented_epochs_train_filtered_downsampled
y_train_filtered_downsampled = augmented_epoch_labels_train_filtered_downsampled_moderate

X_test_filtered_downsampled = X_test_filtered_downsampled_padded
y_test_filtered_downsampled = epoch_labels_test_filtered_downsampled_moderate

# Print shapes

print(f"X_train shape: {X_train_filtered_downsampled.shape}")
print(f"y_train shape: {y_train_filtered_downsampled.shape}")

print(f"\nX_test shape: {X_test_filtered_downsampled.shape}")
print(f"y_test shape: {y_test_filtered_downsampled.shape}")

X_train shape: (20406, 360, 1)
y_train shape: (20406,)

X_test shape: (11440, 360, 1)
y_test shape: (11440,)


#### Train the model

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,               
    restore_best_weights=True 
)
# stop after 5 epochs with no improvement

In [ ]:
# show model architecture
input_shape = (360, 1)
model_filtered_downsampled = build_cnn_model_downsampled(input_shape)
model_filtered_downsampled.summary()

In [ ]:
%%time

# change it to training_info_filtered_downsampled_so next time I run the code

training_info_filtered_downsampled = model_filtered_downsampled.fit(X_train_filtered_downsampled, y_train_filtered_downsampled, validation_split=0.2, epochs=30, batch_size=128, callbacks=[early_stop])

#### Plot the training history

In [ ]:
def plot_training_history(training_info):
    fig, axs = plt.subplots(1, 2, figsize=(16, 5))
    axs[0].plot(training_info.history['loss'], label="training set")
    axs[0].plot(training_info.history['val_loss'], label="validation set")
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].grid(True)
    axs[0].legend()
    try:
        axs[1].plot(training_info.history['accuracy'], label="training set")
        axs[1].plot(training_info.history['val_accuracy'], label="validation set")
        axs[1].set_xlabel("Epoch")
        axs[1].set_ylabel("Accuracy")
        axs[1].grid(True)
        axs[1].legend()
    except:
        pass
  
    fig.suptitle("Training history for one-input CNN model with filtered EEG data for SO detection", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

plot_training_history(training_info_filtered_downsampled)

In [ ]:
model_filtered_downsampled.evaluate(X_test_filtered_downsampled, y_test_filtered_downsampled)

In [ ]:
y_pred = model_filtered_downsampled.predict(X_test_raw_downsampled)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test_raw_downsampled, y_pred_labels))
print(classification_report(y_test_raw_downsampled, y_pred_labels))

In [ ]:
# Get predictions
y_pred = model_filtered_downsampled.predict(X_test_raw_downsampled)
y_pred_labels = (y_pred > 0.5).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test_filtered_downsampled, y_pred_labels)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Classification report as a dataframe
report = classification_report(y_test_filtered_downsampled, y_pred_labels, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Confusion matrix plotted as heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix for SO Detection \nusing a one-input CNN with filtered EEG data")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()

# Classification report as a table
fig, ax = plt.subplots(figsize=(10, 4))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=report_df.round(2).values,
                 colLabels=report_df.columns,
                 rowLabels=report_df.index,
                 cellLoc='center',
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)
plt.title("Classification Report for SO Detection \nusing a one-input CNN with filtered EEG data", fontsize=14)
plt.tight_layout()
plt.show()


### Epoch the data

### STFT on augmented epochs

In [41]:
fs = train_raw_downsampled.info['sfreq']  
# smaller nperseg means higher time resolution
# noverlap must be less than nperseg

# I want to focus more on the frequency resolution
# at the expense of the time resolution
# for spindles, want a 1-2 Hz resolution (frequency bins every 1-2 Hz)
# let's start with 2 Hz first 
nperseg = 50
noverlap = nperseg // 2
# common practice is to set noverlap to 50% of nperseg


augmented_epochs_train_stft_transformed_downsampled = []

for epoch in augmented_epochs_train_raw_downsampled:
    signal = epoch.squeeze()  # so that has shape (360,)
    if signal.ndim != 1:
        raise ValueError(f"Expected 1D signal, got shape {signal.shape}")
    f, t, Zxx = stft(signal, fs=fs, nperseg=nperseg, noverlap=noverlap)
    spectrogram = np.abs(Zxx)  
    augmented_epochs_train_stft_transformed_downsampled.append(spectrogram)

augmented_epochs_test_stft_transformed_downsampled = []

for epoch in X_test_filtered_downsampled_padded:
    signal = epoch.squeeze()  # so that has shape (360,)
    if signal.ndim != 1:
        raise ValueError(f"Expected 1D signal, got shape {signal.shape}")
    f, t, Zxx = stft(signal, fs=fs, nperseg=nperseg, noverlap=noverlap)
    spectrogram = np.abs(Zxx)  
    augmented_epochs_test_stft_transformed_downsampled.append(spectrogram)
    
# convert into numpy arrays
augmented_epochs_train_stft_transformed_downsampled = np.array(augmented_epochs_train_stft_transformed_downsampled)
augmented_epochs_test_stft_transformed_downsampled = np.array(augmented_epochs_test_stft_transformed_downsampled)


print("Train STFT shape:", augmented_epochs_train_stft_transformed_downsampled.shape)
print("Test STFT shape:", augmented_epochs_test_stft_transformed_downsampled.shape)

# shape is number of epochs, frequency_bins, time_bins


Train STFT shape: (20406, 26, 16)
Test STFT shape: (11440, 26, 16)


#### Define x and y sets

In [42]:
# Define X and y sets and reshape

X_train_stft_downsampled = augmented_epochs_train_stft_transformed_downsampled[..., np.newaxis]  # Shape: (14550, 65, 25, 1)
y_train_stft_downsampled = augmented_epoch_labels_train_raw_downsampled_moderate

X_test_stft_downsampled = augmented_epochs_test_stft_transformed_downsampled[..., np.newaxis]    # Shape: (11440, 65, 25, 1)
y_test_stft_downsampled = augmented_epoch_labels_train_raw_downsampled_moderate
                                                                 

# Print shapes

print(f"X_train shape: {X_train_stft_downsampled.shape}")
print(f"y_train shape: {y_train_stft_downsampled.shape}")

print(f"\nX_test shape: {X_test_stft_downsampled.shape}")
print(f"y_test shape: {y_test_stft_downsampled.shape}")

X_train shape: (20406, 26, 16, 1)
y_train shape: (20406,)

X_test shape: (11440, 26, 16, 1)
y_test shape: (20406,)


In [43]:
# this scales the spectrogram in range 0,1
# this is min-max normalisation

X_train_stft_norm_downsampled = np.array([
    (epoch - np.min(epoch)) / (np.max(epoch) - np.min(epoch) + 1e-8)
    for epoch in X_train_stft_downsampled
])

X_test_stft_norm_downsampled = np.array([
    (epoch - np.min(epoch)) / (np.max(epoch) - np.min(epoch) + 1e-8)
    for epoch in X_test_stft_downsampled
])

In [44]:
# should have values between 0 and 1 

print("Before normalisation:")
print("Max train value:", np.max(X_train_stft_downsampled))
print("Min train value:", np.min(X_train_stft_downsampled))

print("Max test value:", np.max(X_test_stft_downsampled))
print("Min test value:", np.min(X_test_stft_downsampled))

print("\nAfter normalisation:")
print("Max train value:", np.max(X_train_stft_norm_downsampled))
print("Min train value:", np.min(X_train_stft_norm_downsampled))

print("Max test value:", np.max(X_test_stft_norm_downsampled))
print("Min test value:", np.min(X_test_stft_norm_downsampled))

Before normalisation:
Max train value: 863.6970095033798
Min train value: 0.0
Max test value: 598.8204811919833
Min test value: 0.0

After normalisation:
Max train value: 0.9999999999884218
Min train value: 0.0
Max test value: 0.9999999999833005
Min test value: 0.0


## Three-input model with augmented data

In [45]:
X_train_stft_freq = np.mean(X_train_stft_norm_downsampled, axis=1) 
X_test_stft_freq = np.mean(X_test_stft_norm_downsampled, axis=1)

print(f"Shape of X_train_stft_freq:{X_train_stft_freq.shape}")
print(f"Shape of X_test_stft_freq:{X_test_stft_freq.shape}")

Shape of X_train_stft_freq:(20406, 16, 1)
Shape of X_test_stft_freq:(11440, 16, 1)


In [46]:
X_train_dict_freq_augmented = {
    'raw_input': X_train_raw_downsampled,
    'filtered_input': X_train_filtered_downsampled,
    'stft_input': X_train_stft_freq
}

X_test_dict_freq_augmented = {
    'raw_input': X_test_raw_downsampled,
    'filtered_input': X_test_filtered_downsampled,
    'stft_input': X_test_stft_freq
}

In [193]:
X_train_raw_downsampled.shape

(20406, 360, 1)

In [194]:
X_train_filtered_downsampled.shape

(20406, 360, 1)

In [195]:
X_train_stft_freq.shape

(20406, 16, 1)

In [47]:
cnn_model_multi_input_freq_augmented = build_multi_input_cnn_model_freq_augmented()
cnn_model_multi_input_freq_augmented.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ raw_input           │ (None, 360, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ filtered_input      │ (None, 360, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stft_input          │ (None, 16, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_3    │ (None, 364, 1)    │          0 │ raw_input[0][0]   │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_4    │ (None, 370, 1)    │          0 │ raw_input[0][0]   │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_5    │ (None, 380, 1)    │          0 │ raw_input[0][0]   │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_6    │ (None, 364, 1)    │          0 │ filtered_input[0… │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_7    │ (None, 370, 1)    │          0 │ filtered_input[0… │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_8    │ (None, 380, 1)    │          0 │ filtered_input[0… │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_9    │ (None, 20, 1)     │          0 │ stft_input[0][0]  │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_10   │ (None, 26, 1)     │          0 │ stft_input[0][0]  │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_11   │ (None, 36, 1)     │          0 │ stft_input[0][0]  │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 360, 10)   │         60 │ zero_padding1d_3… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 360, 10)   │        120 │ zero_padding1d_4… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 360, 10)   │        220 │ zero_padding1d_5… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 360, 10)   │         60 │ zero_padding1d_6… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 360, 10)   │        120 │ zero_padding1d_7… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 360, 10)   │        220 │ zero_padding1d_8… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 16, 10)    │         60 │ zero_padding1d_9

 Total params: 69,273 (270.60 KB)

 Trainable params: 69,093 (269.89 KB)

 Non-trainable params: 180 (720.00 B)

In [48]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,               
    restore_best_weights=True 
)
# stop after 5 epochs with no improvement

In [49]:
%%time

training_info_multiple_inputs_freq_augmented = cnn_model_multi_input_freq_augmented.fit(X_train_dict_freq_augmented, y_train_raw_downsampled, validation_split=0.2, epochs=20, batch_size=128, callbacks=[early_stop])

Epoch 1/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 84s 509ms/step - accuracy: 0.7300 - loss: 0.5244 - val_accuracy: 0.7418 - val_loss: 0.5202
Epoch 2/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 64s 500ms/step - accuracy: 0.8437 - loss: 0.3411 - val_accuracy: 0.7940 - val_loss: 0.4083
Epoch 3/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 64s 499ms/step - accuracy: 0.8811 - loss: 0.2546 - val_accuracy: 0.8792 - val_loss: 0.2522
Epoch 4/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 65s 506ms/step - accuracy: 0.8912 - loss: 0.2357 - val_accuracy: 0.9086 - val_loss: 0.2172
Epoch 5/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 65s 507ms/step - accuracy: 0.9141 - loss: 0.1940 - val_accuracy: 0.9199 - val_loss: 0.1940
Epoch 6/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 65s 504ms/step - accuracy: 0.9259 - loss: 0.1733 - val_accuracy: 0.9287 - val_loss: 0.1637
Epoch 7/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 66s 515ms/step - accuracy: 0.9295 - loss: 0.1681 - val_accuracy: 0.9292 - val_loss: 0.1626
Epoch 8/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 68s 532ms/step - accuracy: 0.9268 - loss: 0

In [ ]:
def plot_training_history(training_info):
    fig, axs = plt.subplots(1, 2, figsize=(16, 5))
    axs[0].plot(training_info.history['loss'], label="training set")
    axs[0].plot(training_info.history['val_loss'], label="validation set")
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].grid(True)
    axs[0].legend()
    try:
        axs[1].plot(training_info.history['accuracy'], label="training set")
        axs[1].plot(training_info.history['val_accuracy'], label="validation set")
        axs[1].set_xlabel("Epoch")
        axs[1].set_ylabel("Accuracy")
        axs[1].grid(True)
        axs[1].legend()
    except:
        pass
  
    fig.suptitle("Training History for three-input CNN model with frequency component of STFT and data augmentation for SO detection", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

plot_training_history(training_info_multiple_inputs_freq)_augmented)

In [ ]:
cnn_model_multi_input_freq_augmented.evaluate(X_test_dict_freq_augmented, y_test_raw_downsampled)

In [50]:
y_pred = cnn_model_multi_input_freq_augmented.predict(X_test_dict_freq_augmented)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test_raw_downsampled, y_pred_labels))
print(classification_report(y_test_raw_downsampled, y_pred_labels))

358/358 ━━━━━━━━━━━━━━━━━━━━ 15s 38ms/step
[[9013  628]
 [ 283 1516]]
              precision    recall  f1-score   support

           0       0.97      0.93      0.95      9641
           1       0.71      0.84      0.77      1799

    accuracy                           0.92     11440
   macro avg       0.84      0.89      0.86     11440
weighted avg       0.93      0.92      0.92     11440



In [ ]:
# Get predictions
y_pred = cnn_model_multi_input_freq_augmented.predict(X_test_dict_freq_agmented)
y_pred_labels = (y_pred > 0.5).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test_raw_downsampled, y_pred_labels)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Classification report as a dataframe
report = classification_report(y_test_raw_downsampled, y_pred_labels, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Confusion matrix plotted as heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix for SO Detection \nusing a three-input CNN with frequency \ncomponent of STFT \nand data augmentation")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.show()

# Classification report as a table
fig, ax = plt.subplots(figsize=(10, 4))
ax.axis('tight')
ax.axis('off')
table = ax.table(cellText=report_df.round(2).values,
                 colLabels=report_df.columns,
                 rowLabels=report_df.index,
                 cellLoc='center',
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)
plt.title("Classification Report for SO Detection \nusing a three-input CNN with frequency \ncomponent of STFT \nand data augmentation", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
# Flatten in case y_pred has shape (n_samples, 1)
y_pred_proba = y_pred.ravel()

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test_raw_downsampled, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic for a three-input CNN model \nwith the STFT frequency component for SO detection \and data augmentation")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()